In [ ]:
import json
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Data Conversion

## Users

In [ ]:
# Read the JSON file
with open('users.json', 'r') as file:
    json_data = [json.loads(line) for line in file]

# Convert JSON data to DataFrame
users = pd.json_normalize(json_data)

# Flatten nested JSON fields
users['_id'] = users['_id.$oid']
users['createdDate'] = pd.to_datetime(users['createdDate.$date'], unit='ms')
if 'lastLogin.$date' in users.columns:
    users['lastLogin'] = pd.to_datetime(users['lastLogin.$date'], unit='ms')

# Drop original nested columns
users.drop(columns=['_id.$oid', 'createdDate.$date', 'lastLogin.$date'], inplace=True, errors='ignore')

users['lastLogin'].replace('', pd.NA, inplace=True)

# Save the updated DataFrame back to a CSV file
users.to_csv('users.csv', index=False, na_rep='NULL')

print("Users JSON data has been converted to CSV successfully.")

Users JSON data has been converted to CSV successfully.


In [ ]:
users.head()

,active,role,signUpSource,state,_id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872


## Brands

In [ ]:
# Read the JSON file
with open('brands.json', 'r') as file:
    json_data = [json.loads(line) for line in file]

# Convert JSON data to DataFrame
brands = pd.json_normalize(json_data)

# Flatten nested JSON fields
brands['_id'] = brands['_id.$oid']
brands['cpg_id'] = brands['cpg.$id.$oid']
brands['cpg_ref'] = brands['cpg.$ref']

# Drop original nested columns
brands.drop(columns=['_id.$oid', 'cpg.$id.$oid', 'cpg.$ref'], inplace=True, errors='ignore')

# Save DataFrame to CSV
brands.to_csv('brands.csv', index=False)

print("Brands JSON data has been converted to CSV successfully.")

Brands JSON data has been converted to CSV successfully.


In [ ]:
brands.head()

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,False,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs


## Receipts

In [ ]:
def read_json(filename: str) -> list:
    with open(filename, "r") as f:
        try:
            # Attempt to load the entire file content as a list of JSON objects
            content = f.read()
            data_list = [json.loads(obj) for obj in content.split('\n') if obj.strip()]
            return data_list
        except json.JSONDecodeError as e:
            raise Exception(f"Reading {filename} file encountered an error: {e}")

# Read the JSON file
try:
    data_list = read_json("receipts.json")
    print("Data Size:", len(data_list))
except Exception as e:
    print(e)

Data Size: 1119


In [ ]:
def normalize_json(data: dict) -> dict:
    new_data = {}
    for key, value in data.items():
        if not isinstance(value, dict):
            new_data[key] = value
        else:
            for k, v in value.items():
                new_data[key + "_" + k] = v
    return new_data

# Normalize each JSON object
normalized_data = [normalize_json(data) for data in data_list]

# Create a pandas dataframe from the list of normalized JSON objects
receipts = pd.DataFrame(normalized_data)

# Write to a CSV file
receipts.to_csv("receipts.csv", index=False)

In [ ]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id_$oid                 1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate_$date         1119 non-null   int64  
 4   dateScanned_$date        1119 non-null   int64  
 5   finishedDate_$date       568 non-null    float64
 6   modifyDate_$date         1119 non-null   int64  
 7   pointsAwardedDate_$date  537 non-null    float64
 8   pointsEarned             609 non-null    object 
 9   purchaseDate_$date       671 non-null    float64
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    object 
 14  userId                  

In [ ]:
receipts.describe()

,bonusPointsEarned,createDate_$date,dateScanned_$date,finishedDate_$date,modifyDate_$date,pointsAwardedDate_$date,purchaseDate_$date,purchasedItemCount
count,544.000000,1.119000e+03,1.119000e+03,5.680000e+02,1.119000e+03,5.370000e+02,6.710000e+02,635.00000
mean,238.893382,1.611800e+12,1.611800e+12,1.611058e+12,1.611847e+12,1.610949e+12,1.608548e+12,14.75748
std,299.091731,1.484091e+09,1.484091e+09,9.534641e+08,1.361576e+09,1.056718e+09,1.230755e+10,61.13424
min,5.000000,1.604089e+12,1.604089e+12,1.609687e+12,1.609687e+12,1.604089e+12,1.509322e+12,0.00000
25%,5.000000,1.610652e+12,1.610652e+12,1.610141e+12,1.610660e+12,1.610043e+12,1.609805e+12,1.00000
50%,45.000000,1.611941e+12,1.611941e+12,1.611091e+12,1.611941e+12,1.611090e+12,1.610551e+12,2.00000
75%,500.000000,1.612704e+12,1.612704e+12,1.611769e+12,1.612704e+12,1.611710e+12,1.611624e+12,5.00000
max,750.000000,1.614641e+12,1.614641e+12,1.614379e+12,1.614641e+12,1.614379e+12,1.615225e+12,689.00000


In [ ]:
def parse_rewards_receipt_item_list(item):
    if isinstance(item, list):
        return item  # Item is already a list
    if isinstance(item, float):
        return []  # Return an empty list for NaN values
    try:
        return json.loads(item.replace("'", '"'))
    except (json.JSONDecodeError, TypeError):
        return []


# Create a copy of df with only the necessary columns
df_subset = receipts[['_id_$oid', 'userId', 'rewardsReceiptItemList']].copy()

# Explode the rewardsReceiptItemList to rows
df_subset_exploded = df_subset.explode('rewardsReceiptItemList').dropna()

# Normalize the rewardsReceiptItemList
items = pd.json_normalize(df_subset_exploded['rewardsReceiptItemList'])

# Merge userId and _id_$oid into the normalized items
items = items.join(df_subset_exploded[['_id_$oid', 'userId']].reset_index(drop=True))

# Display the first few rows of the resulting DataFrame
items.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,needsFetchReviewReason,pointsNotAwardedReason,pointsPayerId,rewardsGroup,rewardsProductPartnerId,userFlaggedDescription,originalMetaBriteBarcode,originalMetaBriteDescription,brandCode,competitorRewardsGroup,discountedItemPrice,originalReceiptItemText,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,_id_$oid,userId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,10.00,1.0,USER_FLAGGED,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6


In [ ]:
items.to_csv('items.csv', index=False)

In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   barcode                             3090 non-null   object 
 1   description                         6560 non-null   object 
 2   finalPrice                          6767 non-null   object 
 3   itemPrice                           6767 non-null   object 
 4   needsFetchReview                    813 non-null    object 
 5   partnerItemId                       6941 non-null   object 
 6   preventTargetGapPoints              358 non-null    object 
 7   quantityPurchased                   6767 non-null   float64
 8   userFlaggedBarcode                  337 non-null    object 
 9   userFlaggedNewItem                  323 non-null    object 
 10  userFlaggedPrice                    299 non-null    object 
 11  userFlaggedQuantity                 299 non

In [ ]:
items.describe()

,quantityPurchased,userFlaggedQuantity,originalMetaBriteQuantityPurchased
count,6767.000000,299.000000,15.000000
mean,1.386139,1.872910,1.200000
std,1.204363,1.314823,0.414039
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000
max,17.000000,5.000000,2.000000


In [ ]:
brands['brandCode'].unique()

array([nan, 'STARBUCKS', 'TEST BRANDCODE @1612366146176',
       'TEST BRANDCODE @1612366146051', 'TEST BRANDCODE @1612366146827',
       'TEST BRANDCODE @1612366146091', 'TEST BRANDCODE @1612366146133',
       'J.L. KRAFT', 'CAMPBELLS HOME STYLE', 'TEST',
       'TEST BRANDCODE @1598813526777', 'CALUMET', '511111205012',
       'AUNT JEMIMA SYRUP', 'MOLSON', 'LOTRIMIN',
       'TEST BRANDCODE @1597342520277', 'ST IVES', 'CHRISIMAGE',
       'ALKA SELTZER', "JACK DANIEL'S BARBECUE", 'MAGNUM Ice Cream',
       '511111105329', 'TEST BRANDCODE @1598635634882', 'TACO BELL',
       'FROSTED CHEERIOS', 'TEST BRANDCODE @1598639199674',
       'GODIVA DRY PACKAGED DESSERTS', 'LARABAR',
       'TEST BRANDCODE @1597350074333', 'TEST BRANDCODE @1607636368717',
       'TEST BRANDCODE @1607707830095', 'COTTONELLE', 'IZZE', 'MIO',
       '511111505365', 'QUILTING SPECIAL EDITION',
       'TEST BRANDCODE @1604437351617', 'HERMAN', 'KEVITA', 'DELIMEX',
       'THE RIGHT TO SHOWER', 'CARESS', 'TEST BRA

In [ ]:
b = items['brandCode'].dropna()
b.unique()

array(['MISSION', 'BRAND', 'KRAFT EASY CHEESE', 'PEPSI', 'DORITOS',
       'KLEENEX', 'WINGSTOP', 'GERM-X', 'BEN AND JERRYS', 'BORDEN',
       'KNORR', 'KLARBRUNN', 'HY-VEE', 'LIGHT & FIT GREEK',
       "CONNIE'S PIZZA", "VAN DE KAMP'S", 'HATCH FARMS', "KELLOGG'S",
       'TEMPTATIONS', "NATURE'S PATH ORGANIC", 'DOLE', 'EL MONTEREY',
       'BIGELOW', 'HY-VEE SELECT', 'KIKKOMAN', 'SPECIAL K', 'SWANSON',
       'YUBAN', 'HILLSHIRE FARM', 'JUST BARE', "LAURA'S LEAN BEEF",
       'CAL-ORGANIC FARMS', 'DOLE CHILLED FRUIT JUICES', "BUSH'S BEST",
       'FOLGERS', 'KASHI', 'LIPTON', 'KRAFT', 'GREEN GIANT',
       'HARVEST SNAPS', "THAT'S SMART!", 'TOSTITOS', 'ADVIL',
       'CHICKEN OF THE SEA', 'RICE-A-RONI', 'STARKIST', 'TIC TAC',
       'SO DELICIOUS', 'WONDERFUL', 'LIGHT & FIT', 'HANOVER',
       'HIDDEN VALLEY', 'DANNON', 'KETTLE BRAND', 'FAGE', 'ORAL-B GLIDE',
       "CAMPBELL'S", "FRENCH'S", 'CRISPIX', 'KING ARTHUR FLOUR',
       'KITCHEN BASICS', 'MCCORMICK', 'OLD EL PASO', 'PEPPERID

In [ ]:
intersect = [item for item in brands['brandCode'].unique() if item in b.unique()]
intersect

['TACO BELL',
 'COTTONELLE',
 'SWANSON',
 'KETTLE BRAND',
 'DORITOS',
 'KLONDIKE',
 'PLANTERS',
 'CHEETOS',
 'COOL WHIP',
 'TOSTITOS',
 'NATURE VALLEY',
 'GREY POUPON',
 'PACIFIC FOODS',
 'KRAFT',
 'PEPPERIDGE FARM',
 'QUAKER',
 'OSCAR MAYER',
 'YUBAN',
 'SARGENTO',
 'KNORR',
 'FINISH',
 'JELL-O',
 'RICE-A-RONI',
 'CRACKER BARREL',
 'LUNCHABLES',
 'HUGGIES',
 'VELVEETA',
 'JUST CRACK AN EGG',
 'PEPSI',
 "HELLMANN'S/BEST FOODS",
 'DOLE CHILLED FRUIT JUICES',
 'CLASSICO',
 'ARNOLD',
 'MOUNTAIN DEW',
 'V8',
 'PREGO',
 'PHILADELPHIA',
 'KLEENEX',
 'VIVA',
 'ORE-IDA',
 'STOVE TOP']

# Data Cleaning

In [ ]:
users = pd.read_csv('users.csv')
brands = pd.read_csv('brands.csv')
receipts = pd.read_csv('receipts.csv')
items_df = pd.read_csv('items.csv')

## Users

In [ ]:
users.head()

,active,role,signUpSource,state,_id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
1,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
2,True,consumer,Email,WI,5ff1e1e8cfcf6c399c274ad9,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392000000
3,True,consumer,Email,WI,5ff1e1b7cfcf6c399c274a5a,2021-01-03 15:24:39.626,2021-01-03 15:24:39.664999936
4,True,consumer,Email,WI,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599000064


### Users Data Schema
- _id: user Id
- state: state abbreviation
- createdDate: when the user created their account
- lastLogin: last time the user was recorded logging in to the app
- role: constant value set to 'CONSUMER'
- active: indicates if the user is active; only Fetch will de-activate an account with this flag

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   active        495 non-null    bool  
 1   role          495 non-null    object
 2   signUpSource  447 non-null    object
 3   state         439 non-null    object
 4   _id           495 non-null    object
 5   createdDate   495 non-null    object
 6   lastLogin     433 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


In [ ]:
users.isna().sum()

active           0
role             0
signUpSource    48
state           56
_id              0
createdDate      0
lastLogin       62
dtype: int64

In [ ]:
users.describe(include='all')

,active,role,signUpSource,state,_id,createdDate,lastLogin
count,495,495,447,439,495,495,433
unique,2,2,2,8,212,212,172
top,True,consumer,Email,WI,54943462e4b07e684157a532,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204000000
freq,494,413,443,396,20,20,20


- Checking Value Counts for some columns

In [ ]:
for col in ['active', 'role', 'signUpSource']:
    print(f"{col}: {users[col].value_counts()}")

active: active
True     494
False      1
Name: count, dtype: int64
role: role
consumer       413
fetch-staff     82
Name: count, dtype: int64
signUpSource: signUpSource
Email     443
Google      4
Name: count, dtype: int64


- Just One Inactive user, let's see that user

In [ ]:
users[users['active'] == False]

,active,role,signUpSource,state,_id,createdDate,lastLogin
240,False,consumer,Email,WI,6008622ebe5fc9247bab4eb9,2021-01-20 17:02:38.662,2021-01-20 17:02:38.931000064


- Since there are duplicate user IDs, let's see if the same user appears again in the table or not.

In [ ]:
users[users['_id'] == '6008622ebe5fc9247bab4eb9']

,active,role,signUpSource,state,_id,createdDate,lastLogin
240,False,consumer,Email,WI,6008622ebe5fc9247bab4eb9,2021-01-20 17:02:38.662,2021-01-20 17:02:38.931000064


- What is fetch-staff? Since the data description says that it is consumer by default

In [ ]:
users[users['role'] == 'fetch-staff']

,active,role,signUpSource,state,_id,createdDate,lastLogin
395,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
396,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
397,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
398,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
399,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
400,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
401,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
402,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
403,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
405,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936


- Missing Signup sources assumption: They are primarily fetch-staff.
- Conclusion: False. there are some consumers as well.

In [ ]:
missing_signupsource = users.groupby('role')['signUpSource'].apply(lambda x: x.isna().sum())
print(missing_signupsource)

role
consumer        9
fetch-staff    39
Name: signUpSource, dtype: int64


- How many signup sources from Google?

In [ ]:
users[users['signUpSource'] == 'Google']

,active,role,signUpSource,state,_id,createdDate,lastLogin
170,True,consumer,Google,WI,5e27526d0bdb6a138c32b556,2020-01-21 19:35:09.795,NaN
420,True,consumer,Google,AL,5fb0a078be5fc9775c1f3945,2020-11-15 03:28:56.818,NaN
429,True,consumer,Google,AL,5fb0a078be5fc9775c1f3945,2020-11-15 03:28:56.818,NaN
456,True,fetch-staff,Google,AL,5fa32b4d898c7a11a6bcebce,2020-11-04 22:29:33.309,2021-03-04 07:21:58.047000064


- Correcting the Date columns into proper format: '%Y-%m-%d %H:%M:%S.%f' and filling the lastlogin column using 2 strategies:
1. If a user ID is repeated, check if other rows of the same user ID have a lastlogin value, if yes, use that value.
2. If a user IDs all instances have lastlogin as NULL, fill the value with the created date as the user must be logged in for the first time when the account was created.

In [ ]:
# Convert the 'createdDate' and 'lastLogin' columns to datetime format
users['createdDate'] = pd.to_datetime(users['createdDate'], format='%Y-%m-%d %H:%M:%S.%f')
users['lastLogin'] = pd.to_datetime(users['lastLogin'], format='%Y-%m-%d %H:%M:%S.%f')

# Fill missing 'lastLogin' values with corresponding 'createdDate' for each '_id'
users['lastLogin'] = users.groupby('_id')['lastLogin'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
users['lastLogin'] = users['lastLogin'].fillna(users['createdDate'])

users.head()

,active,role,signUpSource,state,_id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   active        495 non-null    bool          
 1   role          495 non-null    object        
 2   signUpSource  447 non-null    object        
 3   state         439 non-null    object        
 4   _id           495 non-null    object        
 5   createdDate   495 non-null    datetime64[ns]
 6   lastLogin     495 non-null    datetime64[ns]
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 23.8+ KB


- Filling the missing values in singUpSource and state using the same strategy 1 as lastLogin. But none of the cells got filled using this strategy
- I won't drop any other missing values since we can work around those.

In [ ]:
users['signUpSource'] = users.groupby('_id')['signUpSource'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
users['state'] = users.groupby('_id')['state'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   active        495 non-null    bool          
 1   role          495 non-null    object        
 2   signUpSource  447 non-null    object        
 3   state         439 non-null    object        
 4   _id           495 non-null    object        
 5   createdDate   495 non-null    datetime64[ns]
 6   lastLogin     495 non-null    datetime64[ns]
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 23.8+ KB


- Final Missing Value Rows in Users:

In [ ]:
missing = users[users.isna().any(axis=1)]
missing[:20]

,active,role,signUpSource,state,_id,createdDate,lastLogin
344,True,consumer,Email,NaN,60145ff384231211ce796d51,2021-01-29 19:20:19.722,2021-01-29 19:20:19.722000000
350,True,consumer,Email,NaN,60145ff384231211ce796d51,2021-01-29 19:20:19.722,2021-01-29 19:20:19.722000000
375,True,consumer,Email,NaN,60186237c8b50e11d8454d5f,2021-02-01 20:19:03.551,2021-02-01 20:19:03.551000000
376,True,consumer,Email,NaN,60186237c8b50e11d8454d5f,2021-02-01 20:19:03.551,2021-02-01 20:19:03.551000000
378,True,consumer,Email,NaN,60186237c8b50e11d8454d5f,2021-02-01 20:19:03.551,2021-02-01 20:19:03.551000000
381,True,consumer,Email,NaN,60186237c8b50e11d8454d5f,2021-02-01 20:19:03.551,2021-02-01 20:19:03.551000000
382,True,consumer,Email,NaN,60186237c8b50e11d8454d5f,2021-02-01 20:19:03.551,2021-02-01 20:19:03.551000000
388,True,consumer,NaN,WI,55308179e4b0eabd8f99caa2,2015-04-17 03:43:53.186,2018-05-07 17:23:40.003000064
395,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
396,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936


- Checking for duplicated rows

In [ ]:
users.duplicated().sum()

283

- Dropping duplicate Rows

In [ ]:
cleaned_users = users.drop_duplicates().reset_index(drop=True)
cleaned_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   active        212 non-null    bool          
 1   role          212 non-null    object        
 2   signUpSource  207 non-null    object        
 3   state         206 non-null    object        
 4   _id           212 non-null    object        
 5   createdDate   212 non-null    datetime64[ns]
 6   lastLogin     212 non-null    datetime64[ns]
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 10.3+ KB


In [ ]:
cleaned_users.head(10)

,active,role,signUpSource,state,_id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
1,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
2,True,consumer,Email,WI,5ff1e1e8cfcf6c399c274ad9,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392000000
3,True,consumer,Email,WI,5ff1e1b7cfcf6c399c274a5a,2021-01-03 15:24:39.626,2021-01-03 15:24:39.664999936
4,True,consumer,Email,WI,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599000064
5,True,consumer,Email,WI,5ff1e1e4cfcf6c399c274ac3,2021-01-03 15:25:24.656,2021-01-03 15:25:24.694000128
6,True,consumer,Email,WI,5ff1e1b4cfcf6c399c274a54,2021-01-03 15:24:36.410,2021-01-03 15:24:36.452000000
7,True,consumer,Email,WI,5ff370c562fde912123a5e0e,2021-01-04 19:47:17.776,2021-01-04 19:50:50.563000064
8,True,consumer,Email,WI,5ff36d0362fde912123a5535,2021-01-04 19:31:15.973,2021-01-04 19:34:42.944000000
9,True,consumer,Email,WI,5ff36d83135e7011bcb864d6,2021-01-04 19:33:23.244,2021-01-04 19:33:23.424999936


In [ ]:
missing = cleaned_users [cleaned_users.isna().any(axis=1)]
missing[:20]

,active,role,signUpSource,state,_id,createdDate,lastLogin
165,True,consumer,Email,NaN,60145ff384231211ce796d51,2021-01-29 19:20:19.722,2021-01-29 19:20:19.722000000
180,True,consumer,Email,NaN,60186237c8b50e11d8454d5f,2021-02-01 20:19:03.551,2021-02-01 20:19:03.551000000
188,True,consumer,NaN,WI,55308179e4b0eabd8f99caa2,2015-04-17 03:43:53.186,2018-05-07 17:23:40.003000064
194,True,fetch-staff,NaN,WI,59c124bae4b0299e55b0f330,2017-09-19 14:07:54.302,2021-02-08 16:42:58.116999936
204,True,consumer,NaN,NaN,5a43c08fe4b014fd6b6a0612,2017-12-27 15:47:27.059,2021-02-12 16:22:37.155000064
206,True,fetch-staff,Email,NaN,5fbc35711d967d1222cbfefc,2020-11-23 22:19:29.509,2021-02-26 04:25:51.056999936
208,True,fetch-staff,Email,NaN,5fa41775898c7a11a6bcef3e,2020-11-05 15:17:09.396,2021-03-04 16:02:02.025999872
210,True,fetch-staff,NaN,IL,5964eb07e4b03efd0c0f267b,2017-07-11 15:13:11.771,2021-03-04 19:07:49.769999872
211,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204000000


In [ ]:
cleaned_users.to_csv('cleaned_users.csv', index=False)

## brands

#### **Brand Data Schema**


- _id: brand uuid
- barcode: the barcode on the item
- brandCode: String that corresponds with the brand column in a partner product file
- category: The category name for which the brand sells products in
- categoryCode: The category code that references a BrandCategory
- cpg: reference to CPG collection
- topBrand: Boolean indicator for whether the brand should be featured as a 'top brand'
- name: Brand name

In [ ]:
cleaned_brands = brands.copy()

In [ ]:
cleaned_brands.head()

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,False,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs


In [ ]:
cleaned_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   int64 
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   brandCode     898 non-null    object
 6   _id           1167 non-null   object
 7   cpg_id        1167 non-null   object
 8   cpg_ref       1167 non-null   object
dtypes: int64(1), object(8)
memory usage: 82.2+ KB


In [ ]:
cleaned_brands.describe(include='all')

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
count,1.167000e+03,1012,517,1167,555,898,1167,1167,1167
unique,NaN,23,14,1156,2,896,1167,196,2
top,NaN,Baking,BAKING,Huggies,False,GOODNITES,601ac115be37ce2ead437551,559c2234e4b06aca36af13c6,Cogs
freq,NaN,369,359,2,524,2,1,98,1020
mean,5.111115e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.874497e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5.111110e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,5.111112e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5.111114e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,5.111117e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cleaned_brands.isna().sum()

barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       269
_id               0
cpg_id            0
cpg_ref           0
dtype: int64

- Checking for duplicate rows

In [ ]:
cleaned_brands.duplicated().sum()

0

- Getting an overview of value distribution of some columns

In [ ]:
for col in ['brandCode', 'name', 'category', 'categoryCode']:
    print(f"{col}: {cleaned_brands[col].nunique()} values: {cleaned_brands[col].value_counts()}")
    print()

brandCode: 896 values: brandCode
GOODNITES                        2
HUGGIES                          2
TEST BRANDCODE @1598711015578    1
SOL                              1
TEST BRANDCODE @1599159969725    1
                                ..
TEST BRANDCODE @1599159969028    1
TEST BRANDCODE @1597350074404    1
SEDAL                            1
RED ROCK DELI                    1
TEST BRANDCODE @1613158231644    1
Name: count, Length: 896, dtype: int64

name: 1156 values: name
Huggies                      2
V8 Hydrate                   2
Pull-Ups                     2
Dippin Dots® Cereal          2
Diabetic Living Magazine     2
                            ..
Claritin® KIDS               1
Athenos                      1
test brand @1599159969028    1
test brand @1597350074404    1
test brand @1613158231643    1
Name: count, Length: 1156, dtype: int64

category: 23 values: category
Baking                         369
Beer Wine Spirits               90
Snacks                          75
C

In [ ]:
cleaned_brands.columns

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', 'brandCode', '_id', 'cpg_id', 'cpg_ref'], dtype='object')

- Making sure values in these columns are consistent and not have extra leading or trailing spaces:

In [ ]:
cleaned_brands['name'] = brands['name'].str.strip()
cleaned_brands['category'] = brands['category'].str.strip()
cleaned_brands['brandCode'] = brands['brandCode'].str.strip().str.upper()

- Checking first 20 rows with missing values

In [ ]:
missing = cleaned_brands[cleaned_brands.isna().any(axis=1)]
missing[:20]

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
7,511111104810,Condiments & Sauces,NaN,J.L. Kraft,NaN,J.L. KRAFT,5cdad0f5166eb33eb7ce0faa,559c2234e4b06aca36af13c6,Cogs
8,511111504412,Canned Goods & Soups,NaN,Campbell's Home Style,False,CAMPBELLS HOME STYLE,5ab15636e4b0be0a89bb0b07,5a734034e4b0d58f376be874,Cogs
9,511111504788,Baking,NaN,test,NaN,TEST,5c408e8bcd244a1fdb47aee7,59ba6f1ce4b092b29c167346,Cogs
10,511111516354,Baking,BAKING,test brand @1598813526777,NaN,TEST BRANDCODE @1598813526777,5f4bf556be37ce0b4491554d,5f4bf556be37ce0b44915549,Cogs
11,511111102540,NaN,NaN,MorningStar,NaN,NaN,57c08106e4b0718ff5fcb02c,5332f5f2e4b03c9a25efd0aa,Cpgs
12,511111201076,Baking,NaN,Calumet,False,CALUMET,588ba07be4b02187f85cdadd,559c2234e4b06aca36af13c6,Cogs
13,511111205012,Magazines,NaN,Entertainment Weekly,NaN,511111205012,5d6413156d5f3b23d1bc790a,5d5d4fd16d5f3b23d1bc7905,Cogs
14,511111801801,Breakfast & Cereal,NaN,AUNT JEMIMA Syrup,False,AUNT JEMIMA SYRUP,585a9611e4b03e62d1ce0e74,5332f5fbe4b03c9a25efd0ba,Cogs
15,511111202233,Beer Wine Spirits,NaN,Molson Canadian,False,MOLSON,57e5820ce4b0ac389136a311,5332f709e4b03c9a25efd0f1,Cpgs


- Checking the topBrand column

In [ ]:
cleaned_brands['topBrand'].value_counts()

topBrand
False    524
True      31
Name: count, dtype: int64

- Hmm, total values don't match the total number of rows. Let's see how many missing rows are there:

In [ ]:
cleaned_brands['topBrand'].isna().sum()

612

- **Assumption**: I am listing NULL topBrands as False. Because if they were important to be noted as Top Brand, the value should have been present.

In [ ]:
cleaned_brands['topBrand'] = brands['topBrand'].fillna(False)

- Type casting to INT for easier import in MySQL. MySQL Requires imported boolean fields to be noted as integers. The field topBrand in the MySQL schema will be denoted as Boolean only

In [ ]:
cleaned_brands['topBrand'] = cleaned_brands['topBrand'].astype(int)
cleaned_brands['topBrand'].value_counts()

topBrand
0    1136
1      31
Name: count, dtype: int64

- So, we have 31 brands as topBrand. Let's check these brands

In [ ]:
top_brands = cleaned_brands[cleaned_brands['topBrand'] == 1]
top_brands

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
58,511111106876,Grocery,NaN,DASH-2249 Brand1,1,TEST BRAND CODE,5c76d3cd95144c5375687b4f,5c76d20595144c5375687b4e,Cogs
109,511111801757,Snacks,NaN,Chester's,1,CHESTER'S,585a9645e4b03e62d1ce0e79,5332f5fbe4b03c9a25efd0ba,Cogs
115,511111001119,Snacks,NaN,Doritos,1,DORITOS,5887a372e4b02187f85cdad9,5332f5fbe4b03c9a25efd0ba,Cogs
116,511111101895,Condiments & Sauces,NaN,A.1.,1,A.1.,57ed0697e4b072ac2294b8f2,559c2234e4b06aca36af13c6,Cogs
152,511111204923,Grocery,NaN,Brand1,1,0987654321,5c45f91b87ff3552f950f027,5c45f8b087ff3552f950f026,Cogs
192,511111001768,Snacks,NaN,Cheetos,1,CHEETOS,585a963ce4b03e62d1ce0e78,5332f5fbe4b03c9a25efd0ba,Cogs
245,511111812449,Magazines,NaN,Test brand1,1,NaN,5d66961cee7f2d201c7281cc,5c4f20b01b4181298aacffc1,Cogs
258,511111902461,Baby,NaN,Antarctica,1,AMP2,57c0827de4b0718ff5fcb037,5332f7a7e4b03c9a25efd134,Cogs
271,511111501770,Breakfast & Cereal,NaN,Cap'n Crunch,1,CAP'N CRUNCH,585a9637e4b03e62d1ce0e77,53e10d6368abd3c7065097cc,Cogs
278,511111801689,Snacks,NaN,Lay's Kettle Cooked,1,NaN,585a967fe4b03e62d1ce0e80,5332f5fbe4b03c9a25efd0ba,Cogs


In [ ]:
cleaned_brands.nunique()

barcode         1160
category          23
categoryCode      14
name            1156
topBrand           2
brandCode        895
_id             1167
cpg_id           196
cpg_ref            2
dtype: int64

In [ ]:
cleaned_brands.head()

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs


In [ ]:
cleaned_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   int64 
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      1167 non-null   int64 
 5   brandCode     898 non-null    object
 6   _id           1167 non-null   object
 7   cpg_id        1167 non-null   object
 8   cpg_ref       1167 non-null   object
dtypes: int64(2), object(7)
memory usage: 82.2+ KB


In [ ]:
cleaned_brands.isna().sum()

barcode           0
category        155
categoryCode    650
name              0
topBrand          0
brandCode       269
_id               0
cpg_id            0
cpg_ref           0
dtype: int64

- **Assumption**: Categories have corresponding category code. Since a lot of category codes are missing, we can fill those codes with corresponding category.

In [ ]:
missing

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
7,511111104810,Condiments & Sauces,NaN,J.L. Kraft,NaN,J.L. KRAFT,5cdad0f5166eb33eb7ce0faa,559c2234e4b06aca36af13c6,Cogs
8,511111504412,Canned Goods & Soups,NaN,Campbell's Home Style,False,CAMPBELLS HOME STYLE,5ab15636e4b0be0a89bb0b07,5a734034e4b0d58f376be874,Cogs
9,511111504788,Baking,NaN,test,NaN,TEST,5c408e8bcd244a1fdb47aee7,59ba6f1ce4b092b29c167346,Cogs
10,511111516354,Baking,BAKING,test brand @1598813526777,NaN,TEST BRANDCODE @1598813526777,5f4bf556be37ce0b4491554d,5f4bf556be37ce0b44915549,Cogs
...,...,...,...,...,...,...,...,...,...
1161,511111403845,Beer Wine Spirits,NaN,Blue Moon,False,BLUE MOON,5332f709e4b03c9a25efd0f2,5332f709e4b03c9a25efd0f1,Cogs
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs


- We can see that category code for row 9 and 10 is null but the category is baking. We see row 0 has baking as category and as the category code.
- We can use this information to fill the missing category code in row 9 and 10

In [ ]:
def fill_with_mode(series):
    if series.mode().empty:
        return series
    else:
        return series.fillna(series.mode()[0])

cleaned_brands['categoryCode'] = cleaned_brands.groupby('category')['categoryCode'].transform(fill_with_mode)

In [ ]:
cleaned_brands.isna().sum()

barcode           0
category        155
categoryCode    368
name              0
topBrand          0
brandCode       269
_id               0
cpg_id            0
cpg_ref           0
dtype: int64

- Some rows did get filled. Let's check the first 15 rows and focus on row 9 and 10

In [ ]:
cleaned_brands.head(10)

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs
5,511111719885,Baking,BAKING,test brand @1612366146091,0,TEST BRANDCODE @1612366146091,601ac142be37ce2ead43755b,601ac142be37ce2ead437559,Cogs
6,511111219897,Baking,BAKING,test brand @1612366146133,0,TEST BRANDCODE @1612366146133,601ac142be37ce2ead43755c,601ac142be37ce2ead437559,Cogs
7,511111104810,Condiments & Sauces,NaN,J.L. Kraft,0,J.L. KRAFT,5cdad0f5166eb33eb7ce0faa,559c2234e4b06aca36af13c6,Cogs
8,511111504412,Canned Goods & Soups,NaN,Campbell's Home Style,0,CAMPBELLS HOME STYLE,5ab15636e4b0be0a89bb0b07,5a734034e4b0d58f376be874,Cogs
9,511111504788,Baking,BAKING,test,0,TEST,5c408e8bcd244a1fdb47aee7,59ba6f1ce4b092b29c167346,Cogs


- We can see the cells are filled as we planned!

In [ ]:
cleaned_brands.to_csv('cleaned_brands.csv', index=False)

## Receipts

#### **Receipts Data Schema**
- _id: uuid for this receipt
- bonusPointsEarned: Number of bonus points that were awarded upon receipt completion
- bonusPointsEarnedReason: event that triggered bonus points
- createDate: The date that the event was created
- dateScanned: Date that the user scanned their receipt
- finishedDate: Date that the receipt finished processing
- modifyDate: The date the event was modified
- pointsAwardedDate: The date we awarded points for the transaction
- pointsEarned: The number of points earned for the receipt
- purchaseDate: the date of the purchase
- purchasedItemCount: Count of number of items on the receipt
- rewardsReceiptItemList: The items that were purchased on the receipt
- rewardsReceiptStatus: status of the receipt through receipt validation and processing
- totalSpent: The total amount on the receipt
- userId: string id back to the User collection for the user who scanned the receipt

In [ ]:
cleaned_receipts = receipts.copy()
cleaned_receipts.head()

,_id_$oid,bonusPointsEarned,bonusPointsEarnedReason,createDate_$date,dateScanned_$date,finishedDate_$date,modifyDate_$date,pointsAwardedDate_$date,pointsEarned,purchaseDate_$date,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [ ]:
cleaned_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id_$oid                 1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate_$date         1119 non-null   int64  
 4   dateScanned_$date        1119 non-null   int64  
 5   finishedDate_$date       568 non-null    float64
 6   modifyDate_$date         1119 non-null   int64  
 7   pointsAwardedDate_$date  537 non-null    float64
 8   pointsEarned             609 non-null    float64
 9   purchaseDate_$date       671 non-null    float64
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [ ]:
cleaned_receipts.describe(include='all')

,_id_$oid,bonusPointsEarned,bonusPointsEarnedReason,createDate_$date,dateScanned_$date,finishedDate_$date,modifyDate_$date,pointsAwardedDate_$date,pointsEarned,purchaseDate_$date,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
count,1119,544.000000,544,1.119000e+03,1.119000e+03,5.680000e+02,1.119000e+03,5.370000e+02,609.000000,6.710000e+02,635.00000,679,1119,684.000000,1119
unique,1119,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,383,5,NaN,258
top,5ff1e1eb0a720f0523000575,NaN,All-receipts receipt bonus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'description': 'flipbelt level terrain waist...,FINISHED,NaN,5fc961c3b8cfca11a077dd33
freq,1,NaN,183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,518,NaN,436
mean,NaN,238.893382,NaN,1.611800e+12,1.611800e+12,1.611058e+12,1.611847e+12,1.610949e+12,585.962890,1.608548e+12,14.75748,NaN,NaN,77.796857,NaN
std,NaN,299.091731,NaN,1.484091e+09,1.484091e+09,9.534641e+08,1.361576e+09,1.056718e+09,1357.166947,1.230755e+10,61.13424,NaN,NaN,347.110349,NaN
min,NaN,5.000000,NaN,1.604089e+12,1.604089e+12,1.609687e+12,1.609687e+12,1.604089e+12,0.000000,1.509322e+12,0.00000,NaN,NaN,0.000000,NaN
25%,NaN,5.000000,NaN,1.610652e+12,1.610652e+12,1.610141e+12,1.610660e+12,1.610043e+12,5.000000,1.609805e+12,1.00000,NaN,NaN,1.000000,NaN
50%,NaN,45.000000,NaN,1.611941e+12,1.611941e+12,1.611091e+12,1.611941e+12,1.611090e+12,150.000000,1.610551e+12,2.00000,NaN,NaN,18.200000,NaN
75%,NaN,500.000000,NaN,1.612704e+12,1.612704e+12,1.611769e+12,1.612704e+12,1.611710e+12,750.000000,1.611624e+12,5.00000,NaN,NaN,34.960000,NaN


In [ ]:
def remove_date_suffix(df):
  """
  Removes the `$date` suffix from column names that end with it.

  Args:
      df: The pandas DataFrame to modify.

  Returns:
      A new DataFrame with the modified column names.
  """

  new_columns = []
  for column in df.columns:
    if column.endswith('_$date'):
      new_columns.append(column[:-6])
    else:
      new_columns.append(column)

  df.columns = new_columns
  return df

def clean_receipts(df):
  # Remove the $oid from column name
  df.rename(columns={"_id_$oid": "_id"}, inplace=True)
  df = remove_date_suffix(df)

  # Cast the date fields into datetime data type - similar to how it was done for users
  date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
  for col in date_columns:
      df[col] = pd.to_datetime(df[col], unit='ms', errors='coerce')
      df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
      df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S.%f')

  # Due to some import problems in MySQL, this was a safeguard step to avoid fatal import erros.
  # Filled the null values in bonusPointsEarned and TotalSpent columns to 0
  # Casted the data type as float - the data was already float but there were some fatal errors during data import in mySQL
  for col in ['bonusPointsEarned', 'totalSpent']:
      df[col] = df[col].fillna(0.0)
      df[col] = df[col].astype(float)

clean_receipts(cleaned_receipts)
cleaned_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [ ]:
cleaned_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   _id                      1119 non-null   object        
 1   bonusPointsEarned        1119 non-null   float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             568 non-null    datetime64[ns]
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        537 non-null    datetime64[ns]
 8   pointsEarned             609 non-null    float64       
 9   purchaseDate             671 non-null    datetime64[ns]
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptItemList   679 non-null    object        
 12  rewardsReceiptStatus     1119 non-

In [ ]:
cleaned_receipts.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'pointsEarned', 'purchaseDate', 'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent', 'userId'], dtype='object')

In [ ]:
cleaned_receipts.head(20)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
5,5ff1e1e40a7214ada1000566,750.0,"Receipt number 1 completed, bonus point schedu...",2021-01-03 15:25:24,2021-01-03 15:25:24,2021-01-03 15:25:25,2021-01-03 15:25:30,2021-01-03 15:25:25,750.0,2021-01-02 15:25:24,1.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,3.25,5ff1e1e4cfcf6c399c274ac3
6,5ff1e1cd0a720f052300056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:01,2021-01-03 15:25:01,2021-01-03 15:25:02,2021-01-03 15:25:02,2021-01-03 15:25:02,5.0,2021-01-03 15:25:01,1.0,"[{'brandCode': 'MISSION', 'competitorRewardsGr...",FINISHED,2.23,5ff1e194b6a9d73a3a9f1052
7,5ff1e1a40a720f0523000569,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:24:20,2021-01-03 15:24:20,2021-01-03 15:24:21,2021-01-03 15:24:21,2021-01-03 15:24:21,500.0,2020-12-27 00:00:00,1.0,"[{'barcode': '046000832517', 'brandCode': 'BRA...",FINISHED,10.00,5ff1e194b6a9d73a3a9f1052
8,5ff1e1ed0a7214ada100056e,5.0,All-receipts receipt bonus,2021-01-03 15:25:33,2021-01-03 15:25:33,2021-01-03 15:25:34,2021-01-03 15:25:38,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,20.00,5ff1e1eacfcf6c399c274ae6
9,5ff1e1eb0a7214ada100056b,250.0,"Receipt number 3 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,250.0,2021-01-03 00:00:00,3.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,20.00,5ff1e1eacfcf6c399c274ae6


- dropping the rewardsReceiptItemList column as we have a dedicated table (items) for this field.

In [ ]:
cleaned_receipts.drop(columns=['rewardsReceiptItemList'], inplace=True)

- Checking duplicate rows

In [ ]:
cleaned_receipts.duplicated().sum()

0

- There are many other data cleaning steps that can be performed on this table, but given the business requirements, this much data cleaning is sufficient. Other cleaning steps can be performed as per the client's/stakeholder's requirements.

In [ ]:
cleaned_receipts.to_csv('cleaned_receipts.csv', index=False)

## Items

- Correcting the name of the ID field similar to receipts

In [ ]:
items.rename(columns={"_id_$oid": "_id"}, inplace=True)

In [ ]:
items.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,needsFetchReviewReason,pointsNotAwardedReason,pointsPayerId,rewardsGroup,rewardsProductPartnerId,userFlaggedDescription,originalMetaBriteBarcode,originalMetaBriteDescription,brandCode,competitorRewardsGroup,discountedItemPrice,originalReceiptItemText,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,_id,userId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,10.00,1.0,USER_FLAGGED,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6


In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   barcode                             3090 non-null   object 
 1   description                         6560 non-null   object 
 2   finalPrice                          6767 non-null   object 
 3   itemPrice                           6767 non-null   object 
 4   needsFetchReview                    813 non-null    object 
 5   partnerItemId                       6941 non-null   object 
 6   preventTargetGapPoints              358 non-null    object 
 7   quantityPurchased                   6767 non-null   float64
 8   userFlaggedBarcode                  337 non-null    object 
 9   userFlaggedNewItem                  323 non-null    object 
 10  userFlaggedPrice                    299 non-null    object 
 11  userFlaggedQuantity                 299 non

In [ ]:
items.nunique()

barcode                                568
description                           1889
finalPrice                             828
itemPrice                              828
needsFetchReview                         2
partnerItemId                          916
preventTargetGapPoints                   1
quantityPurchased                       13
userFlaggedBarcode                       6
userFlaggedNewItem                       1
userFlaggedPrice                        13
userFlaggedQuantity                      5
needsFetchReviewReason                   2
pointsNotAwardedReason                   1
pointsPayerId                           15
rewardsGroup                           182
rewardsProductPartnerId                 16
userFlaggedDescription                   3
originalMetaBriteBarcode                 6
originalMetaBriteDescription             2
brandCode                              227
competitorRewardsGroup                  30
discountedItemPrice                    817
originalRec

In [ ]:
items.duplicated().sum()

0

- There won't be any cleaning with this table as the fields are not necessarily present for each receipt and user.
- The rows that had all the field values as NULL except the user and receipt IDs were already dropped during the table creation stage.
- For example: Not every item has a coupon applied to it, which results in a preiceAfterCoupon field for that particular item in the receipt.
- This table is created for easier access of fields required for the queries.

In [ ]:
items.to_csv('items.csv', index=False)

# Exploring the Data before Queries

In [ ]:
brand_names = cleaned_brands['brandCode'].unique()
brand_names

array([nan, 'STARBUCKS', 'TEST BRANDCODE @1612366146176',
       'TEST BRANDCODE @1612366146051', 'TEST BRANDCODE @1612366146827',
       'TEST BRANDCODE @1612366146091', 'TEST BRANDCODE @1612366146133',
       'J.L. KRAFT', 'CAMPBELLS HOME STYLE', 'TEST',
       'TEST BRANDCODE @1598813526777', 'CALUMET', '511111205012',
       'AUNT JEMIMA SYRUP', 'MOLSON', 'LOTRIMIN',
       'TEST BRANDCODE @1597342520277', 'ST IVES', 'CHRISIMAGE',
       'ALKA SELTZER', "JACK DANIEL'S BARBECUE", 'MAGNUM ICE CREAM',
       '511111105329', 'TEST BRANDCODE @1598635634882', 'TACO BELL',
       'FROSTED CHEERIOS', 'TEST BRANDCODE @1598639199674',
       'GODIVA DRY PACKAGED DESSERTS', 'LARABAR',
       'TEST BRANDCODE @1597350074333', 'TEST BRANDCODE @1607636368717',
       'TEST BRANDCODE @1607707830095', 'COTTONELLE', 'IZZE', 'MIO',
       '511111505365', 'QUILTING SPECIAL EDITION',
       'TEST BRANDCODE @1604437351617', 'HERMAN', 'KEVITA', 'DELIMEX',
       'THE RIGHT TO SHOWER', 'CARESS', 'TEST BRA

In [ ]:
item_brands_names = items['brandCode'].unique()
item_brands_names

array([nan, 'MISSION', 'BRAND', 'KRAFT EASY CHEESE', 'PEPSI', 'DORITOS',
       'KLEENEX', 'WINGSTOP', 'GERM-X', 'BEN AND JERRYS', 'BORDEN',
       'KNORR', 'KLARBRUNN', 'HY-VEE', 'LIGHT & FIT GREEK',
       "CONNIE'S PIZZA", "VAN DE KAMP'S", 'HATCH FARMS', "KELLOGG'S",
       'TEMPTATIONS', "NATURE'S PATH ORGANIC", 'DOLE', 'EL MONTEREY',
       'BIGELOW', 'HY-VEE SELECT', 'KIKKOMAN', 'SPECIAL K', 'SWANSON',
       'YUBAN', 'HILLSHIRE FARM', 'JUST BARE', "LAURA'S LEAN BEEF",
       'CAL-ORGANIC FARMS', 'DOLE CHILLED FRUIT JUICES', "BUSH'S BEST",
       'FOLGERS', 'KASHI', 'LIPTON', 'KRAFT', 'GREEN GIANT',
       'HARVEST SNAPS', "THAT'S SMART!", 'TOSTITOS', 'ADVIL',
       'CHICKEN OF THE SEA', 'RICE-A-RONI', 'STARKIST', 'TIC TAC',
       'SO DELICIOUS', 'WONDERFUL', 'LIGHT & FIT', 'HANOVER',
       'HIDDEN VALLEY', 'DANNON', 'KETTLE BRAND', 'FAGE', 'ORAL-B GLIDE',
       "CAMPBELL'S", "FRENCH'S", 'CRISPIX', 'KING ARTHUR FLOUR',
       'KITCHEN BASICS', 'MCCORMICK', 'OLD EL PASO', 'PEP

In [ ]:
same_values = set(item_brands_names) & set(brand_names)
same_values

{'ARNOLD',
 'CHEETOS',
 'CLASSICO',
 'COOL WHIP',
 'COTTONELLE',
 'CRACKER BARREL',
 'DOLE CHILLED FRUIT JUICES',
 'DORITOS',
 'FINISH',
 'GREY POUPON',
 "HELLMANN'S/BEST FOODS",
 'HUGGIES',
 'JELL-O',
 'JUST CRACK AN EGG',
 'KETTLE BRAND',
 'KLEENEX',
 'KLONDIKE',
 'KNORR',
 'KRAFT',
 'LUNCHABLES',
 'MOUNTAIN DEW',
 'NATURE VALLEY',
 'ORE-IDA',
 'OSCAR MAYER',
 'PACIFIC FOODS',
 'PEPPERIDGE FARM',
 'PEPSI',
 'PHILADELPHIA',
 'PLANTERS',
 'PREGO',
 'QUAKER',
 'RICE-A-RONI',
 'SARGENTO',
 'STOVE TOP',
 'SWANSON',
 'TACO BELL',
 'TOSTITOS',
 'V8',
 'VELVEETA',
 'VIVA',
 'YUBAN',
 nan}

In [ ]:
cleaned_receipts.iloc[11]['totalSpent'].dtype

dtype('float64')

In [ ]:
cleaned_brands.head(20)

,barcode,category,categoryCode,name,topBrand,brandCode,_id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs
5,511111719885,Baking,BAKING,test brand @1612366146091,0,TEST BRANDCODE @1612366146091,601ac142be37ce2ead43755b,601ac142be37ce2ead437559,Cogs
6,511111219897,Baking,BAKING,test brand @1612366146133,0,TEST BRANDCODE @1612366146133,601ac142be37ce2ead43755c,601ac142be37ce2ead437559,Cogs
7,511111104810,Condiments & Sauces,NaN,J.L. Kraft,0,J.L. KRAFT,5cdad0f5166eb33eb7ce0faa,559c2234e4b06aca36af13c6,Cogs
8,511111504412,Canned Goods & Soups,NaN,Campbell's Home Style,0,CAMPBELLS HOME STYLE,5ab15636e4b0be0a89bb0b07,5a734034e4b0d58f376be874,Cogs
9,511111504788,Baking,BAKING,test,0,TEST,5c408e8bcd244a1fdb47aee7,59ba6f1ce4b092b29c167346,Cogs


In [ ]:
cleaned_receipts.to_csv('cleaned_receipts.csv', index=False)

In [ ]:
cleaned_receipts.describe()

,bonusPointsEarned,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,totalSpent
count,1119.000000,1119,1119,568,1119,537,609.000000,671,635.00000,1119.000000
mean,116.137623,2021-01-28 02:09:41.600271616,2021-01-28 02:09:41.600272384,2021-01-19 12:10:05.020589568,2021-01-28 15:14:28.703043840,2021-01-18 05:42:47.657355776,585.962890,2020-12-21 10:48:45.994038784,14.75748,47.554111
min,0.000000,2020-10-30 20:17:59,2020-10-30 20:17:59,2021-01-03 15:24:10,2021-01-03 15:24:10,2020-10-30 20:18:00,0.000000,2017-10-30 00:00:00,0.00000,0.000000
25%,0.000000,2021-01-14 19:13:03.690499840,2021-01-14 19:13:03.690499840,2021-01-08 21:22:42.500000,2021-01-14 21:32:25.500000,2021-01-07 18:15:47,5.000000,2021-01-05 00:00:00,1.00000,0.000000
50%,0.000000,2021-01-29 17:18:22,2021-01-29 17:18:22,2021-01-19 21:13:57.500000,2021-01-29 17:18:47,2021-01-19 20:54:02,150.000000,2021-01-13 15:18:59,2.00000,1.000000
75%,45.000000,2021-02-07 13:20:13.736999936,2021-02-07 13:20:13.736999936,2021-01-27 17:42:13.500000,2021-02-07 13:20:13.736999936,2021-01-27 01:12:24,750.000000,2021-01-26 01:12:58,5.00000,25.000000
max,750.000000,2021-03-01 23:17:34.772000,2021-03-01 23:17:34.772000,2021-02-26 22:36:25,2021-03-01 23:17:34.772000,2021-02-26 22:36:25,10199.800000,2021-03-08 17:37:13,689.00000,4721.950000
std,240.243665,NaN,NaN,NaN,NaN,NaN,1357.166947,NaN,61.13424,273.944502
